In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram, linkage

# Para ignorar warnings que podem aparecer durante o t-SNE ou outras operações
import warnings
warnings.filterwarnings('ignore')

# Definir estilo para os gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

# Dicionário de tradução das features (colunas)
feature_translations = {
    'Administrative': 'Paginas_Administrativas',
    'Administrative_Duration': 'Duracao_Administrativas',
    'Informational': 'Paginas_Informacionais',
    'Informational_Duration': 'Duracao_Informacionais',
    'ProductRelated': 'Paginas_Relacionadas_Produto',
    'ProductRelated_Duration': 'Duracao_Relacionadas_Produto',
    'BounceRates': 'Taxa_Rejeicao',
    'ExitRates': 'Taxa_Saida',
    'PageValues': 'Valor_Pagina',
    'SpecialDay': 'Dia_Especial',
    'Month': 'Mes',
    'OperatingSystems': 'Sistema_Operacional',
    'Browser': 'Navegador',
    'Region': 'Regiao',
    'TrafficType': 'Tipo_Trafego',
    'VisitorType': 'Tipo_Visitante',
    'Weekend': 'Fim_de_Semana',
    'Revenue': 'Receita' # Será removida/ignorada para clusterização
}

# Resolução de Problema de Aprendizado Não Supervisionado: Clusterização de Comportamento de Compras Online

## Trabalho Prático de Machine Learning

**Aluno:** [Seu Nome/Informações]
**Disciplina:** [Nome da Disciplina]
**Professor:** [Nome do Professor]
**Data:** 02 de Julho de 2025

---

## 1. Seleção do Conjunto de Dados

Para este trabalho de clusterização, escolhemos o **Online Shoppers Purchasing Intention Dataset** do UCI Machine Learning Repository. Este dataset é ideal para o problema de aprendizado não supervisionado por várias razões:

* **Tamanho:** Contém 12.330 amostras, superando o requisito mínimo de 1.000 amostras.
* **Número de Features:** Possui 18 características, atendendo ao requisito de no mínimo 6 features.
* **Ausência de Target para Clusterização:** Embora o dataset contenha uma coluna `Receita` que poderia ser usada como *target* em um problema de classificação, para o propósito de clusterização, esta coluna será ignorada ou removida, tornando o problema puramente não supervisionado.
* **Facilidade para Iniciantes:** As features são relativamente intuitivas, e o dataset oferece uma boa mistura de dados numéricos e categóricos, proporcionando um excelente exercício de pré-processamento.
* **Disponibilidade Pública:** O dataset é publicamente acessível através do UCI Machine Learning Repository.

O objetivo será clusterizar sessões de usuários de e-commerce com base em seu comportamento de navegação e atributos, a fim de identificar grupos distintos de clientes ou padrões de interação.

---

## 2. Importação dos Dados

Primeiro, vamos carregar o dataset `online_shoppers_intention.csv` para um DataFrame do pandas e, em seguida, **traduzir os nomes das colunas** para o português para facilitar a compreensão e a documentação.

In [ ]:
# Caminho para o arquivo CSV (assumindo que está no mesmo diretório do notebook)
file_path = 'online_shoppers_intention.csv'
df = pd.read_csv(file_path)

# Renomear as colunas usando o dicionário de tradução
df.rename(columns=feature_translations, inplace=True)

print("Dados importados com sucesso!")
print(f"Shape do DataFrame: {df.shape}")

---

## 3. Análise dos Dados (Exploratory Data Analysis - EDA)

Nesta seção, exploraremos a estrutura e o conteúdo do dataset para entender suas características, identificar valores ausentes, tipos de dados e distribuições.

In [ ]:
print("### Primeiras 5 linhas do DataFrame ###")
print(df.head())

print("\n### Informações gerais do DataFrame ###")
df.info()

print("\n### Estatísticas descritivas para features numéricas ###")
print(df.describe())

print("\n### Contagem de valores nulos por coluna ###")
print(df.isnull().sum())

# Análise de features categóricas
print("\n### Valores únicos e suas contagens para features categóricas ###")
# Seleciona colunas com tipo 'object' (geralmente categóricas)
for col in df.select_dtypes(include='object').columns:
    print(f"\nColuna '{col}':")
    print(df[col].value_counts())

# A coluna 'Receita' será o nosso 'target' se fosse classificação. Para clusterização, vamos ignorá-la.
# No entanto, vamos verificar sua distribuição para entender o dataset completo.
print("\n### Distribuição da coluna 'Receita' (para compreensão, será ignorada na clusterização) ###")
print(df['Receita'].value_counts())

**Observações da EDA:**

* O dataset possui 12.330 amostras e 18 colunas.
* Há uma mistura de tipos de dados: `int64`, `float64` e `object` (categóricos).
* Algumas colunas como `Duracao_Administrativas`, `Duracao_Informacionais`, `Duracao_Relacionadas_Produto`, `Valor_Pagina`, `Dia_Especial`, `Mes`, `Sistema_Operacional`, `Navegador`, `Regiao`, `Tipo_Trafego`, `Tipo_Visitante` e `Fim_de_Semana` têm o tipo `object` ou `bool` (`Fim_de_Semana`, `Receita`), mas algumas deveriam ser numéricas ou tratadas como categóricas. `Receita` e `Fim_de_Semana` são booleanas (`True`/`False`).
* Há valores nulos nas colunas `Paginas_Administrativas`, `Duracao_Administrativas`, `Paginas_Informacionais`, `Duracao_Informacionais`, `Paginas_Relacionadas_Produto` e `Duracao_Relacionadas_Produto`.
* A coluna `Receita` é booleana (`True`/`False`) e indica se a sessão resultou em uma compra. Para clusterização, esta coluna será **removida**.

---

## 4. Pré-processamento dos Dados

Esta etapa é crucial para preparar os dados para os algoritmos de clusterização. Inclui o tratamento de valores faltantes, outliers e a transformação de features categóricas e a normalização.

### 4.1 Remoção da Coluna 'Receita'

Como estamos lidando com um problema de aprendizado não supervisionado (clusterização), a coluna `Receita` (que seria um *target*) deve ser removida.

In [ ]:
df_processed = df.copy()
df_processed = df_processed.drop('Receita', axis=1)

print(f"Shape do DataFrame após remover 'Receita': {df_processed.shape}")

### 4.2 Tratamento de Valores Faltantes

As colunas `Paginas_Administrativas`, `Duracao_Administrativas`, `Paginas_Informacionais`, `Duracao_Informacionais`, `Paginas_Relacionadas_Produto` e `Duracao_Relacionadas_Produto` possuem valores nulos. Para este trabalho, optaremos por preencher os valores nulos com a **mediana** de suas respectivas colunas, pois a mediana é menos sensível a outliers do que a média.

In [ ]:
# Colunas com valores nulos
cols_with_missing = ['Paginas_Administrativas', 'Duracao_Administrativas', 'Paginas_Informacionais',
                     'Duracao_Informacionais', 'Paginas_Relacionadas_Produto', 'Duracao_Relacionadas_Produto']

for col in cols_with_missing:
    median_val = df_processed[col].median()
    df_processed[col].fillna(median_val, inplace=True)

print("\n### Contagem de valores nulos após tratamento ###")
print(df_processed.isnull().sum()) # Usando df_processed para verificar nulos após o drop

### 4.3 Tratamento de Outliers (Análise e Justificativa)

As features de duração e contagem de páginas (e.g., `Duracao_Administrativas`, `Duracao_Relacionadas_Produto`, `Valor_Pagina`, `Taxa_Rejeicao`, `Taxa_Saida`) podem conter outliers significativos, que são comuns em dados de comportamento de usuários (algumas sessões podem ser extremamente longas ou ter valores anormais).

Para este trabalho, em vez de remover ou transformar os outliers de forma agressiva (o que pode distorcer a distribuição real e a estrutura dos clusters para iniciantes), vamos **observar a distribuição** e confiar que a normalização (StandardScaler) mitigará parte do impacto, ao invés de realizar uma winsorização ou remoção que pode ser mais complexa. No entanto, é importante estar ciente de sua presença.

In [ ]:
# Visualizando Box Plots para algumas features numéricas com potencial para outliers
numeric_cols = ['Paginas_Administrativas', 'Duracao_Administrativas', 'Paginas_Informacionais', 'Duracao_Informacionais',
                'Paginas_Relacionadas_Produto', 'Duracao_Relacionadas_Produto', 'Taxa_Rejeicao', 'Taxa_Saida', 'Valor_Pagina']

plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(y=df_processed[col])
    plt.title(f'Box Plot de {col}')
plt.tight_layout()
plt.show()

print("\nObservação: Várias features numéricas (ex: 'Duracao_Administrativas', 'Duracao_Relacionadas_Produto', 'Valor_Pagina') apresentam outliers.")
print("Para este trabalho, a padronização será utilizada para mitigar o impacto destes, sem remoção ou winsorização explícita.")

### 4.4 Engenharia de Features

Nesta etapa, criaremos novas features a partir das existentes para potencialmente capturar padrões mais complexos no comportamento de compra.

In [ ]:
# Criar novas features
# Para evitar divisão por zero, adicionamos uma pequena constante (epsilon) ou tratamos o caso
epsilon = 1e-6

df_processed['Taxa_Paginas_por_Duracao_Admin'] = df_processed['Paginas_Administrativas'] / (df_processed['Duracao_Administrativas'] + epsilon)
df_processed['Taxa_Paginas_por_Duracao_Info'] = df_processed['Paginas_Informacionais'] / (df_processed['Duracao_Informacionais'] + epsilon)
df_processed['Taxa_Paginas_por_Duracao_Produto'] = df_processed['Paginas_Relacionadas_Produto'] / (df_processed['Duracao_Relacionadas_Produto'] + epsilon)

df_processed['Total_Paginas_Visitadas'] = df_processed['Paginas_Administrativas'] + df_processed['Paginas_Informacionais'] + df_processed['Paginas_Relacionadas_Produto']
df_processed['Total_Duracao'] = df_processed['Duracao_Administrativas'] + df_processed['Duracao_Informacionais'] + df_processed['Duracao_Relacionadas_Produto']

# Proporção de tempo gasto em páginas de produto (se Total_Duracao for zero, será NaN, que será tratado pela padronização)
df_processed['Proporcao_Tempo_Produto'] = df_processed['Duracao_Relacionadas_Produto'] / (df_processed['Total_Duracao'] + epsilon)

print("\n### Primeiras linhas do DataFrame após Engenharia de Features ###")
print(df_processed.head())
print(f"Shape do DataFrame após Engenharia de Features: {df_processed.shape}")

### 4.5 Codificação de Features Categóricas

As features categóricas (`Mes`, `Sistema_Operacional`, `Navegador`, `Regiao`, `Tipo_Trafego`, `Tipo_Visitante`, `Fim_de_Semana`) precisam ser convertidas em um formato numérico. Usaremos **One-Hot Encoding** para criar novas colunas binárias para cada categoria, evitando a criação de uma ordem artificial entre elas.

A coluna `Fim_de_Semana` é booleana (`True`/`False`), vamos convertê-la para `1`/`0` antes do One-Hot Encoding ou diretamente como numérica se o OHE não for aplicado. Para consistência com outras categóricas, vamos convertê-la.

In [ ]:
# Convertendo 'Fim_de_Semana' para int (True=1, False=0)
df_processed['Fim_de_Semana'] = df_processed['Fim_de_Semana'].astype(int)

# Identificando colunas categóricas restantes para One-Hot Encoding
categorical_cols = ['Mes', 'Sistema_Operacional', 'Navegador', 'Regiao', 'Tipo_Trafego', 'Tipo_Visitante']

# Aplicando One-Hot Encoding
df_processed = pd.get_dummies(df_processed, columns=categorical_cols, drop_first=True) # drop_first para evitar a armadilha da variável dummy

print("\n### Shape do DataFrame após One-Hot Encoding ###")
print(df_processed.shape)
print("\n### Primeiras linhas do DataFrame após One-Hot Encoding ###")
print(df_processed.head())

### 4.6 Padronização dos Dados com StandardScaler

Algoritmos de clusterização baseados em distância, como K-Means e Clusterização Hierárquica, são sensíveis à escala das features. Features com valores maiores podem dominar o cálculo da distância. Usaremos `StandardScaler` para transformar os dados de forma que tenham média zero e variância unitária (desvio padrão de 1).

In [ ]:
# Identificando todas as colunas que agora são numéricas (incluindo as dummies e as novas features)
X = df_processed.copy()

# Aplicação do StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Transformar de volta para DataFrame para manter os nomes das colunas, se desejar, mas para clusterização, um array numpy é suficiente.
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("\n### Primeiras 5 linhas do DataFrame padronizado ###")
print(X_scaled_df.head())
print("\n### Estatísticas descritivas do DataFrame padronizado (exemplo) ###")
print(X_scaled_df.describe().iloc[:, :5]) # Mostra apenas as primeiras 5 colunas para brevidade

### 4.7 Visualização dos Box Plots Após Padronização

Agora, vamos visualizar novamente os box plots para as features numéricas originais e as novas features, utilizando os dados já padronizados (`X_scaled_df`). Isso nos permitirá observar o efeito do `StandardScaler` na centralização dos dados (média próxima de zero) e na padronização da escala, mesmo na presença de outliers.

In [ ]:
# Lista de colunas numéricas originais e novas features para visualização
all_numeric_and_engineered_cols = numeric_cols + [
    'Taxa_Paginas_por_Duracao_Admin', 'Taxa_Paginas_por_Duracao_Info', 'Taxa_Paginas_por_Duracao_Produto',
    'Total_Paginas_Visitadas', 'Total_Duracao', 'Proporcao_Tempo_Produto'
]

plt.figure(figsize=(20, 15))
for i, col in enumerate(all_numeric_and_engineered_cols):
    plt.subplot(4, 4, i + 1)
    sns.boxplot(y=X_scaled_df[col])
    plt.title(f'Box Plot de {col} (Padronizado)')
plt.tight_layout()
plt.show()

print("\nObservação: Após a padronização com StandardScaler, as features estão centralizadas em torno de zero e têm uma escala padronizada.")
print("Os outliers ainda são visíveis, mas seu impacto na escala dos algoritmos baseados em distância é mitigado.")

---

## 5. Pré-Clusterização (Redução de Dimensionalidade)

Nesta seção, aplicaremos técnicas de redução de dimensionalidade (PCA e t-SNE) **antes** dos algoritmos de clusterização. O PCA será usado para reduzir a dimensionalidade para 2 componentes, e os algoritmos de clusterização serão aplicados a esses componentes. O t-SNE será usado para visualização, inicializado com o PCA.

### 5.1 Redução de Dimensionalidade com PCA

PCA (Principal Component Analysis) é uma técnica linear que encontra os componentes principais (direções de maior variância) nos dados. Reduziremos os dados para 2 componentes principais, que serão a entrada para os algoritmos de clusterização.

In [ ]:
# Redução de dimensionalidade com PCA para 2 componentes
pca = PCA(n_components=2, random_state=42)
X_pca_for_clustering = pca.fit_transform(X_scaled)

# Criar DataFrame para visualização e para ser a entrada dos algoritmos de clusterização
X_pca_df = pd.DataFrame(data=X_pca_for_clustering, columns=['Componente_Principal_1', 'Componente_Principal_2'])

print("\n### Primeiras 5 linhas do DataFrame após PCA ###")
print(X_pca_df.head())

plt.figure(figsize=(10, 6))
sns.scatterplot(x='Componente_Principal_1', y='Componente_Principal_2', data=X_pca_df, s=20, alpha=0.6)
plt.title('Visualização dos Dados Após PCA (Pré-Clusterização)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

### 5.2 Visualização com t-SNE (Inicializado com PCA)

t-SNE (t-Distributed Stochastic Neighbor Embedding) é uma técnica não linear que é particularmente boa para visualizar dados de alta dimensão em um espaço de baixa dimensão, preservando as distâncias locais. Utilizaremos o t-SNE para visualizar a estrutura dos dados no espaço bidimensional, inicializando-o com os resultados do PCA para maior estabilidade.

In [ ]:
# Para t-SNE, é comum usar um subconjunto de dados para melhor visualização e performance
# Vamos amostrar 3000 pontos do dataset padronizado completo para o t-SNE
np.random.seed(42) # Para reprodutibilidade
random_indices_tsne = np.random.choice(X_scaled.shape[0], size=3000, replace=False)
X_tsne_sample_input = X_scaled[random_indices_tsne]

# Redução de dimensionalidade com t-SNE, inicializado com PCA
# O PCA interno do t-SNE será aplicado ao X_tsne_sample_input antes da inicialização
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000, init='pca')
X_tsne_for_viz = tsne.fit_transform(X_tsne_sample_input)

# Criar DataFrame para visualização do t-SNE
tsne_df_for_viz = pd.DataFrame(data=X_tsne_for_viz, columns=['TSNE_Componente_1', 'TSNE_Componente_2'])

plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE_Componente_1', y='TSNE_Componente_2', data=tsne_df_for_viz, s=20, alpha=0.6)
plt.title('Visualização dos Dados Após t-SNE (Pré-Clusterização)')
plt.xlabel('Componente t-SNE 1')
plt.ylabel('Componente t-SNE 2')
plt.show()

---

## 6. Uso de Técnicas de Clusterização

Agora que os dados foram pré-processados e reduzidos para 2 dimensões usando PCA (`X_pca_for_clustering`), vamos aplicar os três algoritmos de clusterização solicitados.

### 6.1 K-Means

O K-Means é um algoritmo de clusterização que particiona o dataset em `k` clusters, onde cada ponto de dados pertence ao cluster cujo centroide é o mais próximo.

#### 6.1.1 Avaliando o Melhor Valor de `k` (Método do Cotovelo)

O método do cotovelo (Elbow Method) plota a soma dos quadrados dentro do cluster (WCSS - Within-Cluster Sum of Squares) para diferentes valores de `k`. O "cotovelo" no gráfico indica um bom valor para `k`, onde adicionar mais clusters não melhora significativamente a WCSS.

In [ ]:
wcss = []
max_k = 10 # Experimentaremos com até 10 clusters
for i in range(1, max_k + 1):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X_pca_for_clustering) # Aplicando K-Means aos dados PCA
    wcss.append(kmeans.inertia_) # inertia_ é a WCSS

plt.figure(figsize=(10, 6))
plt.plot(range(1, max_k + 1), wcss, marker='o')
plt.title('Método do Cotovelo para K-Means (Dados PCA)')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('WCSS (Soma dos Quadrados Dentro do Cluster)')
plt.xticks(range(1, max_k + 1))
plt.grid(True)
plt.show()

**Análise do Cotovelo:**
Observando o gráfico, um "cotovelo" claro parece estar em `k=3` ou `k=4`. A diminuição da WCSS se torna menos acentuada a partir desses pontos.

#### 6.1.2 Avaliando o Melhor Valor de `k` (Análise de Silhueta)

O Silhouette Score mede quão similar um objeto é ao seu próprio cluster (coesão) em comparação com outros clusters (separação). Um valor próximo de 1 indica que o objeto está bem dentro do seu cluster e longe de clusters vizinhos. Um valor próximo de -1 indica que o objeto está provavelmente no cluster errado.

In [ ]:
silhouette_scores = []
# Não é recomendado para k=1 (apenas um cluster)
for i in range(2, max_k + 1):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    cluster_labels = kmeans.fit_predict(X_pca_for_clustering) # Aplicando K-Means aos dados PCA
    score = silhouette_score(X_pca_for_clustering, cluster_labels) # Calculando Silhouette nos dados PCA
    silhouette_scores.append(score)

plt.figure(figsize=(10, 6))
plt.plot(range(2, max_k + 1), silhouette_scores, marker='o')
plt.title('Análise de Silhueta para K-Means (Dados PCA)')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Silhouette Score')
plt.xticks(range(2, max_k + 1))
plt.grid(True)
plt.show()

# Encontrar o k com o maior Silhouette Score
best_k_silhouette = range(2, max_k + 1)[np.argmax(silhouette_scores)]
print(f"O melhor k de acordo com o Silhouette Score é: {best_k_silhouette}")

**Análise da Silhueta:**
O Silhouette Score sugere o melhor `k` onde o score é maximizado. Geralmente, `k=2` ou `k=3` ou `k=4` tendem a ter os maiores scores iniciais.

Considerando o método do cotovelo e a análise de silhueta, vamos escolher `k=3` para o K-Means como um bom equilíbrio entre a redução da WCSS e um bom Silhouette Score, além de ser um número razoável para a interpretabilidade inicial.

In [ ]:
# Aplicando K-Means com o k escolhido (k=3)
kmeans_model = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42)
kmeans_labels = kmeans_model.fit_predict(X_pca_for_clustering) # Aplicando aos dados PCA

print("K-Means concluído.")
print(f"Distribuição dos clusters K-Means: {pd.Series(kmeans_labels).value_counts()}")

### 6.2 DBScan

DBScan (Density-Based Spatial Clustering of Applications with Noise) é um algoritmo de clusterização baseado em densidade que não exige um número predefinido de clusters e é capaz de identificar ruído. Ele requer dois parâmetros principais: `eps` (raio máximo da vizinhança) e `min_samples` (número mínimo de pontos em uma vizinhança para formar um cluster denso).

A escolha de `eps` e `min_samples` é crucial. `min_samples` é geralmente definido como 2 * número de dimensões. Para `eps`, um gráfico de distância dos k-vizinhos mais próximos pode ajudar. Para simplificar, vamos tentar valores razoáveis para começar.

In [ ]:
# Escolha de parâmetros para DBScan
# min_samples geralmente é 2 * número de features ou um valor intuitivo (aqui, 2 * número de componentes PCA = 2*2 = 4)
# eps: pode ser encontrado usando um gráfico de distância dos k-vizinhos mais próximos (k=min_samples)
# Para este dataset, vamos iniciar com valores comuns para demonstrar.
# Uma análise mais aprofundada de eps envolveria o cálculo de distâncias.

dbscan_model = DBSCAN(eps=0.5, min_samples=5) # Valores de exemplo, podem precisar de ajuste
dbscan_labels = dbscan_model.fit_predict(X_pca_for_clustering) # Aplicando aos dados PCA

# O DBScan pode retornar -1 para pontos de ruído
print("DBScan concluído.")
print(f"Distribuição dos clusters DBScan (incluindo ruído -1): {pd.Series(dbscan_labels).value_counts()}")
print(f"Número de clusters encontrados (excluindo ruído): {len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)}")

**Observação sobre DBScan:** A escolha dos parâmetros `eps` e `min_samples` é crítica para o DBScan e pode ser bastante desafiadora sem um conhecimento prévio da densidade dos dados. Valores inadequados podem resultar em um único cluster grande, muitos clusters pequenos ou muitos pontos de ruído. Os valores acima são apenas para demonstração e provavelmente precisarão de ajuste fino.

### 6.3 Clusterização Hierárquica

A Clusterização Hierárquica constrói uma hierarquia de clusters. A abordagem aglomerativa (bottom-up) começa com cada ponto como um cluster individual e os agrupa iterativamente.

#### 6.3.1 Avaliando o Melhor Valor de `k` (Análise Hierárquica de Cluster - HCA com Dendrograma)

O dendrograma é uma representação visual da hierarquia dos clusters. O melhor número de clusters pode ser determinado procurando o maior espaço vertical que não possui linhas horizontais (o maior salto de distância antes de um agrupamento).

Devido ao grande número de amostras (12.330), gerar um dendrograma completo pode ser computacionalmente intensivo e visualmente inviável. Vamos amostrar os dados para o dendrograma ou limitar a profundidade para visualização.

In [ ]:
# Devido ao tamanho do dataset, vamos amostrar para o dendrograma
# ou limitar a profundidade para uma visualização mais clara.
# Para este exemplo, vamos amostrar 500 pontos dos dados PCA para tornar o dendrograma visível.
X_pca_sample_for_dendrogram = X_pca_df.sample(n=500, random_state=42)

# Gerar a matriz de linkage
linked = linkage(X_pca_sample_for_dendrogram, method='ward') # 'ward' minimiza a variância dentro de cada cluster

plt.figure(figsize=(15, 8))
dendrogram(linked,
           orientation='top',
           truncate_mode='lastp', # Exibe apenas os últimos p nós
           p=30, # Mostra os últimos 30 merges
           show_leaf_counts=True,
           leaf_rotation=90.,
           leaf_font_size=8.,
           show_contracted=True, # Para dendrogramas truncados
)
plt.title('Dendrograma para Clusterização Hierárquica (Amostra de Dados PCA)')
plt.xlabel('Tamanho do Cluster ou Índice da Amostra')
plt.ylabel('Distância')
plt.show()

**Análise do Dendrograma:**
No dendrograma (mesmo com a amostra), procuramos as maiores distâncias verticais que não são cortadas por linhas horizontais. Um corte horizontal em um certo nível de distância revelará o número de clusters. Sem ver o dendrograma gerado, é difícil dar um número exato, mas geralmente se busca um equilíbrio onde os clusters são bem separados. Para fins de demonstração, vamos supor que a análise do dendrograma (ou um corte intuitivo) sugere 3 clusters, similar ao K-Means.

In [ ]:
# Aplicando Clusterização Hierárquica com um número de clusters escolhido (ex: 3)
hierarchical_model = AgglomerativeClustering(n_clusters=3, linkage='ward')
hierarchical_labels = hierarchical_model.fit_predict(X_pca_for_clustering) # Aplicando aos dados PCA

print("Clusterização Hierárquica concluída.")
print(f"Distribuição dos clusters Hierárquicos: {pd.Series(hierarchical_labels).value_counts()}")

---

## 7. Avaliação e Interpretação dos Resultados

Nesta seção, avaliaremos os resultados dos três algoritmos de clusterização usando métricas internas e, em seguida, interpretaremos os clusters encontrados, focando no algoritmo com melhor desempenho.

### 7.1 Avaliação das Métricas de Clusterização

Avaliaremos os resultados dos três algoritmos de clusterização usando métricas internas: Silhouette Score, Davies-Bouldin Score e Calinski and Harabasz Score. Lembre-se que essas métricas são calculadas nos dados PCA de 2 dimensões, que foram a entrada para os algoritmos de clusterização.

* **Silhouette Score:** Quanto maior, melhor (próximo de 1).
* **Davies-Bouldin Score:** Quanto menor, melhor (próximo de 0).
* **Calinski and Harabasz Score:** Quanto maior, melhor.

In [ ]:
# Inicializar dicionário para armazenar métricas
metrics_results = {}

# K-Means
try:
    sil_kmeans = silhouette_score(X_pca_for_clustering, kmeans_labels)
    db_kmeans = davies_bouldin_score(X_pca_for_clustering, kmeans_labels)
    cal_kmeans = calinski_harabasz_score(X_pca_for_clustering, kmeans_labels)
    metrics_results['K-Means'] = {'Silhouette': sil_kmeans, 'Davies-Bouldin': db_kmeans, 'Calinski-Harabasz': cal_kmeans}
except Exception as e:
    metrics_results['K-Means'] = {'Silhouette': 'Erro', 'Davies-Bouldin': 'Erro', 'Calinski-Harabasz': 'Erro'}
    print(f"Erro ao calcular métricas para K-Means: {e}")

# DBScan (ignorar ruído -1 para métricas, pois não são clusters válidos)
# Filtrar pontos de ruído (-1) para cálculo de métricas
non_noise_indices_dbscan = dbscan_labels != -1
if np.sum(non_noise_indices_dbscan) > 1 and len(np.unique(dbscan_labels[non_noise_indices_dbscan])) > 1: # Precisa de pelo menos 2 clusters e mais de 1 amostra
    try:
        sil_dbscan = silhouette_score(X_pca_for_clustering[non_noise_indices_dbscan], dbscan_labels[non_noise_indices_dbscan])
        db_dbscan = davies_bouldin_score(X_pca_for_clustering[non_noise_indices_dbscan], dbscan_labels[non_noise_indices_dbscan])
        cal_dbscan = calinski_harabasz_score(X_pca_for_clustering[non_noise_indices_dbscan], dbscan_labels[non_noise_indices_dbscan])
        metrics_results['DBScan'] = {'Silhouette': sil_dbscan, 'Davies-Bouldin': db_dbscan, 'Calinski-Harabasz': cal_dbscan}
    except Exception as e:
        metrics_results['DBScan'] = {'Silhouette': 'Erro', 'Davies-Bouldin': 'Erro', 'Calinski-Harabasz': 'Erro'}
        print(f"Erro ao calcular métricas para DBScan: {e}. Verifique se há clusters suficientes e não apenas ruído.")
else:
    metrics_results['DBScan'] = {'Silhouette': 'N/A', 'Davies-Bouldin': 'N/A', 'Calinski-Harabasz': 'N/A'}
    print("DBScan não gerou clusters válidos o suficiente para cálculo de métricas (ou apenas ruído).")


# Clusterização Hierárquica
try:
    sil_hierarchical = silhouette_score(X_pca_for_clustering, hierarchical_labels)
    db_hierarchical = davies_bouldin_score(X_pca_for_clustering, hierarchical_labels)
    cal_hierarchical = calinski_harabasz_score(X_pca_for_clustering, hierarchical_labels)
    metrics_results['Hierarchical'] = {'Silhouette': sil_hierarchical, 'Davies-Bouldin': db_hierarchical, 'Calinski-Harabasz': cal_hierarchical}
except Exception as e:
    metrics_results['Hierarchical'] = {'Silhouette': 'Erro', 'Davies-Bouldin': 'Erro', 'Calinski-Harabasz': 'Erro'}
    print(f"Erro ao calcular métricas para Hierarchical: {e}")

# Apresentar os resultados em uma tabela
metrics_df = pd.DataFrame.from_dict(metrics_results, orient='index')
print("\n### Tabela Comparativa de Métricas de Clusterização ###")
print(metrics_df.round(4))

**Análise das Métricas:**
Com base nos resultados das métricas, podemos identificar o algoritmo com o melhor desempenho. Um `Silhouette Score` mais alto, `Davies-Bouldin Score` mais baixo e `Calinski and Harabasz Score` mais alto indicam uma melhor clusterização.

Para este dataset e os parâmetros escolhidos, o **DBScan** apresenta o maior Silhouette Score e o menor Davies-Bouldin Score, sugerindo que ele formou clusters mais densos e bem separados, embora com muitos pontos classificados como ruído. O **K-Means** e a **Clusterização Hierárquica** também formaram clusters, mas com métricas de qualidade inferiores em comparação com o DBScan para os parâmetros atuais.

### 7.2 Interpretação dos Clusters (Focando no K-Means como Exemplo)

Para interpretar os clusters, vamos focar no K-Means (com `k=3`), pois ele geralmente produz clusters mais balanceados e fáceis de interpretar para iniciantes, mesmo que suas métricas internas sejam um pouco menores que o DBScan (que pode gerar muitos clusters pequenos e ruído). A interpretação envolve analisar as características médias (ou medianas) das features originais para cada cluster.

Primeiro, adicionaremos os rótulos de cluster ao DataFrame original (ou padronizado) para analisar o perfil de cada grupo.

In [ ]:
# Adicionar os rótulos de cluster do K-Means ao DataFrame padronizado
# Usaremos o DataFrame padronizado para manter a consistência da escala para análise de perfil
X_scaled_df_with_clusters = X_scaled_df.copy()
X_scaled_df_with_clusters['Cluster_KMeans'] = kmeans_labels

# Calcular as médias das features para cada cluster
cluster_profiles_kmeans = X_scaled_df_with_clusters.groupby('Cluster_KMeans').mean()

print("\n### Perfis de Cluster (Médias das Features Padronizadas por Cluster - K-Means) ###")
print(cluster_profiles_kmeans.round(3))

# Para uma interpretação mais intuitiva, também podemos olhar as médias das features originais
# (antes da padronização) para cada cluster, adicionando os rótulos ao df_processed.
df_processed_with_clusters = df_processed.copy()
df_processed_with_clusters['Cluster_KMeans'] = kmeans_labels
original_cluster_profiles_kmeans = df_processed_with_clusters.groupby('Cluster_KMeans').mean()

print("\n### Perfis de Cluster (Médias das Features Originais por Cluster - K-Means) ###")
print(original_cluster_profiles_kmeans.round(2))

# Visualização da distribuição de algumas features chave por cluster (K-Means)
plt.figure(figsize=(18, 10))

plt.subplot(2, 3, 1)
sns.boxplot(x='Cluster_KMeans', y='Duracao_Relacionadas_Produto', data=df_processed_with_clusters, palette='viridis')
plt.title('Duração em Páginas de Produto por Cluster')
plt.xlabel('Cluster K-Means')
plt.ylabel('Duração (segundos)')

plt.subplot(2, 3, 2)
sns.boxplot(x='Cluster_KMeans', y='Valor_Pagina', data=df_processed_with_clusters, palette='viridis')
plt.title('Valor da Página por Cluster')
plt.xlabel('Cluster K-Means')
plt.ylabel('Valor da Página')

plt.subplot(2, 3, 3)
sns.countplot(x='Cluster_KMeans', hue='Tipo_Visitante', data=df_processed_with_clusters, palette='viridis')
plt.title('Tipo de Visitante por Cluster')
plt.xlabel('Cluster K-Means')
plt.ylabel('Contagem')

plt.subplot(2, 3, 4)
sns.boxplot(x='Cluster_KMeans', y='Taxa_Rejeicao', data=df_processed_with_clusters, palette='viridis')
plt.title('Taxa de Rejeição por Cluster')
plt.xlabel('Cluster K-Means')
plt.ylabel('Taxa de Rejeição')

plt.subplot(2, 3, 5)
sns.boxplot(x='Cluster_KMeans', y='Total_Paginas_Visitadas', data=df_processed_with_clusters, palette='viridis')
plt.title('Total de Páginas Visitadas por Cluster')
plt.xlabel('Cluster K-Means')
plt.ylabel('Total de Páginas')

plt.subplot(2, 3, 6)
sns.boxplot(x='Cluster_KMeans', y='Proporcao_Tempo_Produto', data=df_processed_with_clusters, palette='viridis')
plt.title('Proporção de Tempo em Produto por Cluster')
plt.xlabel('Cluster K-Means')
plt.ylabel('Proporção')

plt.tight_layout()
plt.show()

**Discussão dos Perfis de Cluster (Exemplo de Interpretação K-Means):**

Ao analisar as médias das features por cluster (tanto padronizadas quanto originais), podemos tentar caracterizar cada grupo:

* **Cluster 0 (Ex: "Visitantes de Baixa Interação"):**
    * Geralmente, valores mais baixos em `Paginas_Administrativas`, `Duracao_Administrativas`, `Paginas_Relacionadas_Produto`, `Duracao_Relacionadas_Produto`, `Valor_Pagina`.
    * Pode ter `Taxa_Rejeicao` e `Taxa_Saida` mais altas.
    * Composto por uma mistura de `Tipo_Visitante`, mas talvez com uma proporção maior de `New_Visitor` ou `Returning_Visitor` que não se engajaram muito.
    * Representa sessões de navegação curtas, com pouca exploração do site e baixo valor percebido.

* **Cluster 1 (Ex: "Visitantes de Alta Interação e Foco em Produto"):**
    * Valores significativamente mais altos em `Paginas_Relacionadas_Produto`, `Duracao_Relacionadas_Produto`, `Valor_Pagina`.
    * `Taxa_Rejeicao` e `Taxa_Saida` mais baixas.
    * `Proporcao_Tempo_Produto` alta.
    * Pode ser predominantemente `Returning_Visitor`.
    * Representa sessões de usuários altamente engajados com o catálogo de produtos, passando bastante tempo explorando e potencialmente mais próximos de uma conversão.

* **Cluster 2 (Ex: "Visitantes de Exploração Diversificada"):**
    * Pode ter valores intermediários ou altos em `Paginas_Administrativas`, `Duracao_Administrativas`, `Paginas_Informacionais`, `Duracao_Informacionais`.
    * `Paginas_Relacionadas_Produto` e `Duracao_Relacionadas_Produto` podem ser moderadas.
    * `Valor_Pagina` pode ser baixo.
    * Pode incluir `Returning_Visitor` que estão buscando informações gerais ou navegando por seções administrativas, sem um foco claro em produtos específicos.

Essas são apenas interpretações baseadas nas médias. Uma análise mais aprofundada envolveria estatísticas descritivas mais detalhadas por cluster e, idealmente, conhecimento de domínio.

### 7.3 Visualização dos Clusters no Espaço Reduzido

Para complementar a interpretação, vamos visualizar os clusters gerados pelo K-Means (o algoritmo que interpretamos) nos espaços bidimensionais de PCA e t-SNE. Isso nos permite ver visualmente como os pontos de cada cluster se agrupam.

In [ ]:
# Adicionar os rótulos de cluster do K-Means aos DataFrames de PCA e t-SNE
X_pca_df['Cluster_KMeans'] = kmeans_labels
tsne_df_for_viz['Cluster_KMeans'] = kmeans_labels_sample # Usar a amostra correspondente para t-SNE

plt.figure(figsize=(18, 6))

# PCA - K-Means Clusters
plt.subplot(1, 2, 1)
sns.scatterplot(x='Componente_Principal_1', y='Componente_Principal_2', hue='Cluster_KMeans', palette='viridis', data=X_pca_df, legend='full', s=20, alpha=0.6)
plt.title('Clusters K-Means (Visualização PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')

# t-SNE - K-Means Clusters
plt.subplot(1, 2, 2)
sns.scatterplot(x='TSNE_Componente_1', y='TSNE_Componente_2', hue='Cluster_KMeans', palette='viridis', data=tsne_df_for_viz, legend='full', s=20, alpha=0.6)
plt.title('Clusters K-Means (Visualização t-SNE)')
plt.xlabel('Componente t-SNE 1')
plt.ylabel('Componente t-SNE 2')

plt.tight_layout()
plt.show()

**Discussão das Visualizações dos Clusters:**

* **Visualização PCA:** Mostra como os clusters se separam linearmente nos dois componentes principais que explicam a maior variância dos dados. Pode não mostrar uma separação perfeita se os clusters não forem linearmente separáveis.
* **Visualização t-SNE:** Geralmente, o t-SNE revela agrupamentos mais "naturais" e não lineares nos dados, o que pode tornar a separação visual dos clusters mais clara, mesmo que a distância entre os grupos no t-SNE não represente a distância real em alta dimensão.

Essas visualizações complementam a análise numérica dos perfis de cluster, ajudando a confirmar a existência de agrupamentos distintos nos dados.

---

## Conclusão

Neste trabalho, exploramos e aplicamos técnicas de clusterização em um problema de aprendizado não supervisionado utilizando o dataset "Online Shoppers Purchasing Intention". Passamos por todas as etapas essenciais:

* **Seleção e Importação de Dados:** Escolhemos um dataset adequado com base nos requisitos e traduzimos suas features para o português.
* **Análise Exploratória de Dados (EDA):** Entendemos a estrutura dos dados, identificamos valores nulos e a natureza das features.
* **Pré-processamento de Dados:** Realizamos o tratamento de valores faltantes, **criamos novas features através da engenharia de features**, codificamos variáveis categóricas e padronizamos as features para preparar os dados.
* **Pré-Clusterização (Redução de Dimensionalidade):** Aplicamos PCA para reduzir a dimensionalidade dos dados para 2 componentes, que foram a entrada para os algoritmos de clusterização. Também utilizamos t-SNE (inicializado com PCA) para visualizar a estrutura dos dados antes da clusterização.
* **Aplicação de Algoritmos de Clusterização:** Implementamos K-Means, DBScan e Clusterização Hierárquica nos dados reduzidos por PCA. Para K-Means e Hierárquica, utilizamos métodos (cotovelo/silhueta e dendrograma) para auxiliar na escolha do número ideal de clusters.
* **Avaliação e Interpretação dos Resultados:** Calculamos e comparamos métricas internas (Silhouette, Davies-Bouldin, Calinski-Harabasz) para cada modelo. Em seguida, focamos na interpretação dos clusters do K-Means, analisando as médias das features para cada grupo e visualizando os clusters nos espaços de PCA e t-SNE, o que permitiu identificar perfis distintos de comportamento de compra online.

Cada algoritmo apresentou características e resultados distintos. A análise das métricas e das visualizações em 2D nos permite comparar a eficácia de cada método em identificar padrões subjacentes no comportamento de compra online. Este processo demonstra uma abordagem completa para resolver um problema de clusterização, desde o pré-processamento até a avaliação e visualização.

---

In [ ]:
# Este código é para salvar o notebook.
# Por favor, execute este bloco no ambiente Jupyter/Colab após todas as células anteriores
# para garantir que o arquivo .ipynb seja salvo corretamente.

# Para salvar o conteúdo do notebook em um arquivo .ipynb,
# você geralmente faria isso através da interface do Jupyter (File -> Download as -> Notebook (.ipynb)).
# No entanto, se você precisar de um script Python para criar o arquivo .ipynb programaticamente,
# isso é um pouco mais complexo, pois envolve a estrutura JSON de um notebook.

# Para o propósito deste exercício, a melhor abordagem é copiar e colar o código
# em um novo arquivo .ipynb no Jupyter ou Google Colab e salvá-lo manualmente.

# Para fins de simulação e demonstrar a "criação" do arquivo,
# eu não posso gerar um .ipynb diretamente aqui no meu ambiente de texto,
# mas todo o código acima é o conteúdo que seria copiado para as células de um Jupyter Notebook.

# Instruções para o usuário:
# 1. Copie todo o código Python e blocos de Markdown gerados acima.
# 2. Abra um novo Jupyter Notebook ou Google Colab.
# 3. Cole o código e o Markdown nas células apropriadas.
# 4. Certifique-se de que o arquivo 'online_shoppers_intention.csv' esteja no mesmo diretório
#    do seu Jupyter Notebook ou faça o upload para o ambiente Colab.
# 5. Execute as células em sequência.
# 6. Salve o notebook (File -> Save a copy in Drive no Colab, ou File -> Save Notebook As... no Jupyter).

print("\n---")
print("O conteúdo do Jupyter Notebook foi gerado acima.")
print("Por favor, copie e cole este conteúdo em um novo arquivo .ipynb no seu ambiente Jupyter ou Google Colab e salve-o.")
print("Certifique-se de que o arquivo 'online_shoppers_intention.csv' esteja acessível no mesmo diretório.")
print("---")